In [2]:
import pandas as pd
import itertools

import os
import shutil

from dotenv import load_dotenv
load_dotenv()
os.environ['FORGE_JAR_PATH'] = os.getenv('FORGE_JAR_PATH', '')

### Helper Functions

#### Decklist Generation

In [3]:
def generate_decklists(cards_df):
    """
    Creates all possible Jumpstart decks (two half decks put together), then generates output .dck files for use in Forge

    Args:
        cards_df (DataFrame): Dataframe containing cards categorized into decks

    Returns:
        String: Path to output folder
    """
    deck_names = cards_df['deck'].unique()
    deck_combinations = itertools.combinations(deck_names, 2)
    output_path = os.path.join('output', 'jumpstart')

    for deck1_name, deck2_name in deck_combinations:
        deck1 = cards_df[cards_df['deck'] == deck1_name]
        deck2 = cards_df[cards_df['deck'] == deck2_name]
        deck = pd.concat([deck1, deck2])

        generate_deck_file(deck, deck1_name+" "+deck2_name, output_path)

    return output_path


def generate_deck_file(deck, name='Sample Deck', output_path='output/jumpstart'):
    """
    Saves deck as a .dck file for Forge

    Args:
        deck (DataFrame): Dataframe containing cards in a deck
        name (String): Deck name
        output_path (String): Path to output folder
    """
    os.makedirs(output_path, exist_ok=True)

    with open(os.path.join(output_path, f"{name}.dck"), 'w') as f:
        f.write('[metadata]\n')
        f.write(f'Name={name}\n')
        f.write('[Avatar]\n\n')
        f.write('[Main]\n')
        for _, row in deck.iterrows():
            set_code = row['set_code'] if row['set_code'] else ''
            f.write(f"{row['quantity']} {row['name']}|{set_code}|1\n")

        f.write('[Sideboard]\n\n')
        f.write('[Planes]\n\n')
        f.write('[Schemes]\n\n')
        f.write('[Conspiracy]\n\n')
        f.write('[Dungeon]')


def add_lands(cards_df):
    """
    Adds lands to decks that do not have lands. Tedious to do in Archidekt, so it's automated!

    Args:
        cards_df (DataFrame): Dataframe containing cards categorized into decks

    Returns:
        DataFrame: Dataframe containing cards categorized into decks, with added lands
    """
    deck_names = cards_df['deck'].unique()
    lands = {
        'W': [
            {
                'quantity': '7',
                'name':     'Plains',
                'set_code': 'JMP',
                'tag':      'Land',
            },
            {
                'quantity': '1',
                'name':     'Thriving Heath',
                'set_code': 'JMP',
                'tag':      'Land'
            }
        ],
        'U': [
            {
                'quantity': '7',
                'name':     'Island',
                'set_code': 'JMP',
                'tag':      'Land'
            },
            {
                'quantity': '1',
                'name':     'Thriving Isle',
                'set_code': 'JMP',
                'tag':      'Land'
            }
        ],
        'B': [
            {
                'quantity': '7',
                'name':     'Swamp',
                'set_code': 'JMP',
                'tag':      'Land'
            },
            {
                'quantity': '1',
                'name':     'Thriving Moor',
                'set_code': 'JMP',
                'tag':      'Land'
            }
        ],
        'R': [
            {
                'quantity': '7',
                'name':     'Mountain',
                'set_code': 'JMP',
                'tag':      'Land'
            },
            {
                'quantity': '1',
                'name':     'Thriving Bluff',
                'set_code': 'JMP',
                'tag':      'Land'
            }
        ],
        'G': [
            {
                'quantity': '7',
                'name':     'Forest',
                'set_code': 'JMP',
                'tag':      'Land'
            },
            {
                'quantity': '1',
                'name':     'Thriving Grove',
                'set_code': 'JMP',
                'tag':      'Land'
            }
        ],
    }

    for deck in deck_names:
        deck_df = cards_df[cards_df['deck'] == deck]
        colour = deck_df['colour'].iloc[0]

        if deck_df[deck_df['name'].isin(['Plains', 'Island', 'Swamp', 'Mountain', 'Forest'])].empty:
            lands_df = pd.DataFrame.from_dict(lands[colour])
            lands_df['deck'] = deck

            cards_df = pd.concat([cards_df, lands_df])


    return cards_df


def parse_card(card):
    """
    Parses an individual card line from input CSV

    Args:
        card (String): Individual card line from CSV

    Returns:
        Dictionary: Card line parsed into dictionary
    """
    split_card = card.split()

    card_dict = {
        'quantity': '',
        'name':     '',
        'colour':   '',
        'set_code': '',
        'deck':     '',
        'tag':      '',
    }

    card_dict['quantity'] = split_card[0].split('x')[0]

    step = 0
    for index, word in enumerate(split_card[1:]):
        if step == 0 and '(' not in word:
            card_dict['name'] = card_dict['name'] + ' ' + word
        elif step == 0:
            step = 1

        if step == 1:
            card_dict['set_code'] = card_dict['set_code'] + ' ' + word.strip('()').upper()
            if ')' in word:
                step = 2
                continue

        if step == 2:
            deck_name = word.strip('[]')
            card_dict['deck'] = card_dict['deck'] + ' ' + deck_name
            if ' - ' in card_dict['deck']:
                card_dict['colour'] = card_dict['deck'].split(' - ')[0].strip()
                card_dict['deck'] = card_dict['deck'].split(' - ')[1].strip()
            if ']' in word:
                step = 3
            continue

        if step == 3:
            card_dict['tag'] = word.split(',')[0].strip('^')

    for key in card_dict:
        card_dict[key] = card_dict[key].strip().replace('/', '')

    return card_dict


def parse_decks(cards):
    """
    Parses an input CSV

    Args:
        cards (List): List of card lines from CSV

    Returns:
        DataFrame: All cards processed
    """
    card_list = [parse_card(card) for card in cards]
    cards_df = pd.DataFrame.from_dict(card_list)

    return cards_df

### Running Games

In [4]:
import subprocess
import concurrent.futures
from threading import Lock
import time

In [5]:
# Global lock for thread-safe operations if needed
game_lock = Lock()

def run_game(deck1_name, deck2_name, num_games=1, working_dir=None):
    """
    Run a single game between two decks

    Args:
        deck1_name (str): Name of the first deck
        deck2_name (str): Name of the second deck
        num_games (int): Number of games to run (default 1)
        working_dir (str): Working directory for the Java process

    Returns:
        subprocess.CompletedProcess: Game output
    """
    if working_dir is None:
        working_dir = os.path.dirname(os.environ.get("FORGE_JAR_PATH", ""))

    original_cwd = os.getcwd()
    try:
        if working_dir:
            os.chdir(working_dir)

        game_output = subprocess.run([
            "java", "-jar", os.path.basename(os.environ.get("FORGE_JAR_PATH", "")),
            "sim", "-d",
            os.path.join("JUMPSTART", f"{deck1_name}.dck"),
            os.path.join("JUMPSTART", f"{deck2_name}.dck"),
            "-m", str(num_games),
            "-q"
        ], capture_output=True, text=True, timeout=120)

        return game_output
    finally:
        os.chdir(original_cwd)

def run_games_multithreaded(deck_pairs, num_games_per_pair=10, max_workers=4):
    """
    Run multiple games concurrently using ThreadPoolExecutor

    Args:
        deck_pairs (list): List of tuples containing (deck1_name, deck2_name)
        num_games_per_pair (int): Number of games to run for each deck pair
        max_workers (int): Maximum number of concurrent threads

    Returns:
        list: List of results for each game
    """
    results = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all games to the thread pool
        future_to_decks = {}

        for deck1, deck2 in deck_pairs:
            future = executor.submit(run_game, deck1, deck2, num_games_per_pair)
            future_to_decks[future] = (deck1, deck2)

        # Collect results as they complete
        for future in concurrent.futures.as_completed(future_to_decks):
            deck1, deck2 = future_to_decks[future]
            try:
                result = future.result()
                results.append({
                    'deck1': deck1,
                    'deck2': deck2,
                    'result': result,
                    'success': result.returncode == 0
                })
                print(f"Completed: {deck1} vs {deck2}")
            except Exception as exc:
                print(f"Game {deck1} vs {deck2} generated an exception: {exc}")
                results.append({
                    'deck1': deck1,
                    'deck2': deck2,
                    'result': None,
                    'success': False,
                    'error': str(exc)
                })

    return results

def run_games_batch(deck_pairs, num_games_per_pair=10, max_workers=4, batch_size=None):
    """
    Run games in batches to avoid overwhelming the system

    Args:
        deck_pairs (list): List of tuples containing (deck1_name, deck2_name)
        num_games_per_pair (int): Number of games to run for each deck pair
        max_workers (int): Maximum number of concurrent threads
        batch_size (int): Number of deck pairs to process in each batch (default: max_workers * 2)

    Returns:
        list: List of results for all games
    """
    if batch_size is None:
        batch_size = max_workers * 2

    all_results = []

    for i in range(0, len(deck_pairs), batch_size):
        batch = deck_pairs[i:i + batch_size]
        print(f"Processing batch {i//batch_size + 1}/{(len(deck_pairs) + batch_size - 1)//batch_size}")

        batch_results = run_games_multithreaded(batch, num_games_per_pair, max_workers)
        all_results.extend(batch_results)

        # Optional: Add a small delay between batches to prevent system overload
        time.sleep(0.5)

    return all_results

In [6]:
# Example usage functions

def get_all_deck_combinations():
    """
    Get all possible deck combinations from the output directory
    Each .dck file is a complete deck, so we create pairs of these decks to battle against each other

    Returns:
        list: List of tuples containing (deck1_name, deck2_name) where each name is a complete deck file name (without .dck extension)
    """
    jumpstart_dir = os.path.join('output', 'jumpstart')
    if not os.path.exists(jumpstart_dir):
        return []

    # Get all deck file names (without .dck extension)
    deck_names = [f[:-4] for f in os.listdir(jumpstart_dir) if f.endswith('.dck')]

    # deck_names = [name for name in deck_names if 'living' in name.lower() or 'dragons' in name.lower()]

    # Create all possible combinations of decks to battle against each other
    deck_pairs = []
    for i, deck1 in enumerate(deck_names):
        for deck2 in deck_names[i+1:]:  # Avoid duplicates and self-matches
            deck_pairs.append((deck1, deck2))

    return deck_pairs

def get_sample_deck_combinations(num_combinations=10):
    """
    Get a sample of deck combinations for testing

    Args:
        num_combinations (int): Number of random combinations to return

    Returns:
        list: List of tuples containing (deck1_name, deck2_name)
    """
    import random
    all_combinations = get_all_deck_combinations()

    if len(all_combinations) <= num_combinations:
        return all_combinations

    return random.sample(all_combinations, num_combinations)

def evaluate_all_decks_multithreaded(num_games=10, max_workers=4):
    """
    Evaluate all deck combinations using multithreading

    Args:
        num_games (int): Number of games to run for each deck pair
        max_workers (int): Maximum number of concurrent threads

    Returns:
        list: Results from all games
    """
    deck_pairs = get_all_deck_combinations()
    print(f"Found {len(deck_pairs)} deck combinations to evaluate")

    if not deck_pairs:
        print("No deck combinations found. Make sure decks have been generated first.")
        return []

    print(f"Running {num_games} games per combination with {max_workers} workers...")
    start_time = time.time()

    results = run_games_batch(deck_pairs, num_games, max_workers)

    end_time = time.time()
    print(f"Completed in {end_time - start_time:.2f} seconds")

    # Summary statistics
    successful_games = sum(1 for r in results if r['success'])
    failed_games = len(results) - successful_games

    print(f"Results: {successful_games} successful, {failed_games} failed")

    return results

def evaluate_sample_decks_multithreaded(num_combinations=10, num_games=10, max_workers=4):
    """
    Evaluate a sample of deck combinations using multithreading (useful for testing)

    Args:
        num_combinations (int): Number of random deck combinations to test
        num_games (int): Number of games to run for each deck pair
        max_workers (int): Maximum number of concurrent threads

    Returns:
        list: Results from all games
    """
    deck_pairs = get_sample_deck_combinations(num_combinations)
    print(f"Testing {len(deck_pairs)} random deck combinations")

    if not deck_pairs:
        print("No deck combinations found. Make sure decks have been generated first.")
        return []

    print(f"Running {num_games} games per combination with {max_workers} workers...")
    start_time = time.time()

    results = run_games_batch(deck_pairs, num_games, max_workers)

    end_time = time.time()
    print(f"Completed in {end_time - start_time:.2f} seconds")

    # Summary statistics
    successful_games = sum(1 for r in results if r['success'])
    failed_games = len(results) - successful_games

    print(f"Results: {successful_games} successful, {failed_games} failed")

    return results

def parse_game_results(results):
    """
    Parse game results to extract win rates and statistics, creating one row per deck

    Args:
        results (list): Results from run_games_multithreaded or run_games_batch

    Returns:
        pandas.DataFrame: DataFrame with one row per deck including wins, winrate, and turn count statistics
    """
    from statistics import mode, median

    # First, parse individual game results
    game_data = []

    for result in results:
        if result['success'] and result['result']:
            output = result['result'].stdout
            lines = output.strip().split('\n')

            wins_deck1 = 0
            wins_deck2 = 0
            total_games = 0
            turn_counts = []

            for line in lines:
                if 'game outcome: turn' in line.lower():
                    try:
                        turn_count = int(line.lower().split()[-1])
                        turn_counts.append(turn_count)
                    except (ValueError, IndexError):
                        pass

                if 'won!' in line.lower():
                    total_games += 1
                    if result['deck1'].lower() in line.lower():
                        wins_deck1 += 1
                    elif result['deck2'].lower() in line.lower():
                        wins_deck2 += 1

            # Add data for each deck in this matchup
            if total_games > 0:
                avg_turns = sum(turn_counts) / len(turn_counts) if turn_counts else 0
                median_turns = median(turn_counts) if turn_counts else 0
                try:
                    mode_turns = mode(turn_counts) if turn_counts else 0
                except:
                    mode_turns = turn_counts[0] if turn_counts else 0

                # Add deck1 data
                game_data.append({
                    'deck': result['deck1'],
                    'wins': wins_deck1,
                    'losses': wins_deck2,
                    'total_games': total_games,
                    'avg_turns': avg_turns,
                    'median_turns': median_turns,
                    'mode_turns': mode_turns
                })

                # Add deck2 data
                game_data.append({
                    'deck': result['deck2'],
                    'wins': wins_deck2,
                    'losses': wins_deck1,
                    'total_games': total_games,
                    'avg_turns': avg_turns,
                    'median_turns': median_turns,
                    'mode_turns': mode_turns
                })

    if not game_data:
        return pd.DataFrame()

    # Convert to DataFrame for easier aggregation
    df = pd.DataFrame(game_data)

    # Aggregate by deck name
    deck_summary = df.groupby('deck').agg({
        'wins': 'sum',
        'losses': 'sum',
        'total_games': 'sum',
        'avg_turns': 'mean',
        'median_turns': 'mean',
        'mode_turns': 'mean'
    }).reset_index()

    # Calculate win rate
    deck_summary['winrate'] = deck_summary['wins'] / deck_summary['total_games']
    deck_summary['winrate'] = deck_summary['winrate'].round(4)

    # Round turn statistics
    deck_summary['avg_turns'] = deck_summary['avg_turns'].round(2)
    deck_summary['median_turns'] = deck_summary['median_turns'].round(2)
    deck_summary['mode_turns'] = deck_summary['mode_turns'].round(2)

    # Sort by win rate (descending)
    deck_summary = deck_summary.sort_values('winrate', ascending=False).reset_index(drop=True)

    # Reorder columns for better readability
    deck_summary = deck_summary[['deck', 'wins', 'losses', 'total_games', 'winrate',
                                'avg_turns', 'median_turns', 'mode_turns']]

    print(f"\nDeck Performance Summary:")
    print(f"Total decks analyzed: {len(deck_summary)}")
    print(f"Best performing deck: {deck_summary.iloc[0]['deck']} ({deck_summary.iloc[0]['winrate']:.2%} win rate)")
    print(f"Worst performing deck: {deck_summary.iloc[-1]['deck']} ({deck_summary.iloc[-1]['winrate']:.2%} win rate)")

    return deck_summary

### Process Decks

In [7]:
with open('input/jumpstart.txt', 'r') as file:
    cards = file.readlines()

cards_df = parse_decks(cards)
cards_df = add_lands(cards_df)

assert len(cards_df['deck'].unique()) == cards_df['quantity'].astype(int).sum() // 20, "Deck count does not match expected value (cards/20)"

deck_path = generate_decklists(cards_df)
forge_path = os.getenv("FORGE_DECKS_PATH")

# Clear the folder at FORGE_PATH
if forge_path and os.path.exists(forge_path):
    for filename in os.listdir(forge_path):
        file_path = os.path.join(forge_path, filename)
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)

# Copy contents of deck_path to FORGE_PATH
if forge_path and os.path.exists(deck_path):
    for filename in os.listdir(deck_path):
        src_file = os.path.join(deck_path, filename)
        dst_file = os.path.join(forge_path, filename)
        shutil.copy2(src_file, dst_file)

### Run Games

In [ ]:
original_dir = os.getcwd()

In [36]:
all_results = evaluate_all_decks_multithreaded(num_games=2, max_workers=6)

Found 210 deck combinations to evaluate
Running 2 games per combination with 6 workers...
Processing batch 1/18
Completed: Aggressive Dragons vs Bloodthirsty Dragons
Completed: Aggressive Dragons vs Elvish Dragons
Completed: Aggressive Dragons vs Aggressive Living
Completed: Aggressive Dragons vs Goblins Dragons
Completed: Aggressive Dragons vs Gambling Dragons
Completed: Aggressive Dragons vs Dragons Morbid
Completed: Aggressive Dragons vs Growth Dragons
Completed: Aggressive Dragons vs Growth Living
Completed: Aggressive Dragons vs Living Dragons
Completed: Aggressive Dragons vs Living Bloodthirsty
Completed: Aggressive Dragons vs Living Gambling
Completed: Aggressive Dragons vs Living Elvish
Processing batch 2/18
Completed: Aggressive Dragons vs Living Goblins
Completed: Aggressive Dragons vs Plundering Dragons
Completed: Aggressive Dragons vs Living Morbid
Completed: Aggressive Dragons vs Living Stompy
Completed: Aggressive Dragons vs Living Necromancy
Completed: Aggressive Dragons

In [ ]:
os.chdir(original_dir)
game_df = parse_game_results(all_results)
game_time = time.time()
game_df.to_csv(f'output/game_results_{game_time}.csv', index=False)


Deck Performance Summary:
Total decks analyzed: 21
Best performing deck: Living Elvish (64.58% win rate)
Worst performing deck: Living Gambling (27.66% win rate)


### Analyze Results

In [ ]:
game_results_df = pd.read_csv(f'output/game_results_{game_time}.csv')

In [39]:
game_results_df['deck1'] = game_results_df['deck'].str.split().str[0]
game_results_df['deck2'] = game_results_df['deck'].str.split().str[1]

In [40]:
game_results_df.head()

,deck,wins,losses,total_games,winrate,avg_turns,median_turns,mode_turns,deck1,deck2
0,Living Elvish,31,17,48,0.6458,9.27,9.34,9.00,Living,Elvish
1,Bloodthirsty Dragons,30,17,47,0.6383,8.56,8.48,8.70,Bloodthirsty,Dragons
2,Growth Living,31,19,50,0.6200,9.22,8.88,8.90,Growth,Living
3,Elvish Dragons,28,19,47,0.5957,8.42,8.42,8.11,Elvish,Dragons
4,Growth Dragons,32,22,54,0.5926,8.66,8.78,8.60,Growth,Dragons


In [25]:
deck1_summary = game_results_df.groupby('deck1').agg({
    'wins': 'sum',
    'losses': 'sum',
    'total_games': 'sum'
}).reset_index()

deck1_summary['winrate'] = deck1_summary['wins'] / deck1_summary['total_games']
deck1_summary['winrate'] = deck1_summary['winrate'].round(4)
deck1_summary.rename(columns={'deck1': 'deck'}, inplace=True)

deck2_summary = game_results_df.groupby('deck2').agg({
    'wins': 'sum',
    'losses': 'sum',
    'total_games': 'sum'
}).reset_index()

deck2_summary['winrate'] = deck2_summary['wins'] / deck2_summary['total_games']
deck2_summary['winrate'] = deck2_summary['winrate'].round(4)
deck2_summary.rename(columns={'deck2': 'deck'}, inplace=True)

combined_summary = pd.concat([deck1_summary, deck2_summary], ignore_index=True)
combined_summary = combined_summary.groupby('deck').agg({
    'wins': 'sum',
    'losses': 'sum',
    'total_games': 'sum'
}).reset_index()

combined_summary['winrate'] = combined_summary['wins'] / combined_summary['total_games']
combined_summary['winrate'] = combined_summary['winrate'].round(4)
combined_summary.to_csv('output/combined_deck_summary.csv', index=False)

In [28]:
game_results_df[game_results_df['deck'].str.contains('Dragons')]

,deck,wins,losses,total_games,winrate,avg_turns,median_turns,mode_turns,deck1,deck2
15,Goblins Dragons,95,68,163,0.5828,7.61,7.58,7.74,Goblins,Dragons
34,Elvish Dragons,71,69,140,0.5071,8.25,8.17,8.45,Elvish,Dragons
35,Aggressive Dragons,82,82,164,0.5000,8.09,8.01,8.17,Aggressive,Dragons
40,Bloodthirsty Dragons,76,84,160,0.4750,8.66,8.71,8.38,Bloodthirsty,Dragons
45,Plundering Dragons,74,89,163,0.4540,8.76,8.62,8.69,Plundering,Dragons
49,Necromancy Dragons,69,91,160,0.4312,8.90,8.79,8.71,Necromancy,Dragons
50,Growth Dragons,68,96,164,0.4146,8.26,8.23,8.20,Growth,Dragons
56,Stompy Dragons,59,98,157,0.3758,8.45,8.28,8.15,Stompy,Dragons
59,Dragons Morbid,55,105,160,0.3438,8.41,8.31,8.41,Dragons,Morbid
61,Gambling Dragons,54,108,162,0.3333,7.66,7.59,7.66,Gambling,Dragons
